In [ ]:
%tensorboard 

In [ ]:
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import keras
from keras.callbacks import CSVLogger

import data
from modules.utilites import DataClass

seq_len = 100

In [ ]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas())

# row_data['train'][0] = row_data['train'][0][:1000]
# row_data['train'][1]= row_data['train'][1][:1000]

data.inspect_datas(row_data)

In [ ]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

# Test model

In [ ]:
from models import m_base as test_model

In [ ]:
# Model parametrs
class CN_default(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_default(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = False


class FF_default(DataClass):
    dropout_rate = 0.1


class Optimaser_default(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_default(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_default(DataClass):
    cn = CN_default()
    an = AN_default()
    ff = FF_default()
    optimizer = keras.optimizers.Adam(
        learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999,
        name="Adam",
    )
    training = Trainig_default()


default_pars = Full_default()

In [ ]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=default_pars.training.batch_size,
    seq_len=seq_len,
)
ds_train, ds_val, ds_test = datasets['train'], datasets['val'], datasets[
    'test']
data.inspect_datasets(datasets)

In [ ]:
# Build
model = test_model.build_model()
model.summary()

In [ ]:
import time
class CustomCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        if not batch%50:
            time.sleep(10)
            
class CustomCallback(keras.callbacks.TensorBoard):  
    def on_train_batch_end(self, batch, logs=None):
        super().on_train_batch_end(batch, logs=None)
        if not batch%50:
            time.sleep(10)

In [ ]:
# Train
model.fit(
    ds_train,
    epochs=default_pars.training.epochs,
    validation_data=ds_val,
    callbacks=[
    CustomCallback("callbacks/test1",histogram_freq=1,update_freq=1,),
    ]
)

# Search

In [ ]:
from keras_tuner.tuners import GridSearch
import keras_tuner

import models

In [ ]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
ds_train, ds_val, ds_test = datasets['train'], datasets['val'], datasets[
    'test']
data.inspect_datasets(datasets)

In [ ]:
def configure(hp: keras_tuner.HyperParameters):

    class CN_search(DataClass):
        dilation_steps = hp.Int(
            'cn_layers',
            default=4,
            min_value=3,
            max_value=5,
            step=1,
        )

    class AN_search(DataClass):
        attention_heads = 3
        blocks = 2
        share_weights = hp.Boolean(
            'share_weights',
            default=True,
        )

    class FF_search(DataClass):
        dropout_rate = hp.Float(
            'dropout_rate',
            default=0.1,
            min_value=0,
            max_value=0.5,
            step=0.1,
        )

    lr = hp.Float(
        'lr',
        default=1e-4,
        min_value=1e-6,
        max_value=1e-3,
    )
    beta1 = hp.Float(
        'beta1',
        default=0.9,
        min_value=0.5,
        max_value=1.1,
        step=0.1,
    )
    beta2 = hp.Float(
        'beta1',
        default=0.999,
        min_value=0.5,
        max_value=1.1,
        sampling='log',
    )

    class Full_search(DataClass):
        cn = CN_search()
        an = AN_search()
        ff = FF_search()
        optimizer = keras.optimizers.Adam(
            name="Adam",
            learning_rate=lr,
            beta_1=beta1,
            beta_2=beta2,
        )

    return Full_search()


a = configure(keras_tuner.HyperParameters())
a.Info_expanded

In [ ]:
def search_base_model(hp):
    a = configure(hp)
    model = models.m_base.build_model(**a.Info_expanded)
    return model


In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=search_base_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    directory="search",
    project_name="base",
)

In [ ]:
%load_ext tensorboard

In [ ]:
tuner.search(
    ds_train,
    epochs=2,
    callbacks = [
        keras.callbacks.TensorBoard("search/tb_logs")
    ]
)
